In [36]:
import pandas as pd
# Assuming you have already imported necessary libraries and cleaned your text data as mentioned before
from sklearn.feature_extraction.text import TfidfVectorizer
#svm <- TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
# bert <- TfidfVectorizer
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch

In [24]:
df = pd.read_csv('../data/interest-location.csv')
df

,text,location,interest
0,attempt hari tu act like smart deep tech entre...,NaN,Technology
1,mosti cradle ni clueless whats going malaysias...,NaN,Technology
2,hightech exports country china 942b hong kong ...,NaN,Technology
3,hightech exports country china 942b hong kong ...,NaN,Technology
4,tbh malaysia many tech talents dont actually n...,NaN,Technology
...,...,...,...
28835,smartwatch one better sport ea garmin amazfit ...,Terengganu,Sports
28836,abearfromsea heaven football maracana,Terengganu,Sports
28837,king pele former youth sport minister khairykj...,Terengganu,Sports
28838,love football,Terengganu,Sports


In [25]:
df.isna().sum()

text            1
location    12700
interest        0
dtype: int64

In [26]:
missing_values = df['text'].isna().sum()
print("Number of missing values:", missing_values)

Number of missing values: 1


In [27]:
df.dropna(subset=['text'], inplace=True)
df.isna().sum()

text            0
location    12699
interest        0
dtype: int64

In [28]:
df['text'].fillna("No text available", inplace=True)

In [29]:
# Assuming you have a DataFrame 'df' with 'text' and 'label' columns
X = df['text']  # Features
y = df['interest']  # Labels

# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now you have:
# X_train: Training text data
# y_train: Corresponding training labels
# X_test: Testing text data
# y_test: Corresponding testing labe

In [30]:
tfidf_vectorizer = TfidfVectorizer(max_features=29000, stop_words='english')
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

# SVM

In [31]:
# Initialize and train the SVM classifier
svm_classifier = SVC(kernel='linear')  # You can try different kernels (linear, rbf, etc.)
svm_classifier.fit(tfidf_train, y_train)

SVC(kernel='linear')

In [32]:
y_test

23920           Sports
25159           Sports
12011    Entertainment
25378           Sports
18407        Education
             ...      
28480           Sports
3739        Technology
25273           Sports
12517    Entertainment
25960           Sports
Name: interest, Length: 5768, dtype: object

In [33]:
# Predict on the test set
# y_pred = svm_classifier.predict(tfidf_train)
y_pred

array(['Sports', 'Sports', 'Entertainment', ..., 'Sports',
       'Entertainment', 'Sports'], dtype=object)

In [35]:
# Make predictions on the test data
y_pred = svm_classifier.predict(tfidf_test)

# Evaluate the classifier using the test data
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

    Education       0.95      0.95      0.95      1003
Entertainment       0.96      0.97      0.97      1151
   FoodnDrink       0.97      0.96      0.97       817
     Politics       0.97      0.97      0.97       755
       Sports       0.98      0.98      0.98      1116
   Technology       0.96      0.96      0.96       926

     accuracy                           0.96      5768
    macro avg       0.97      0.96      0.96      5768
 weighted avg       0.96      0.96      0.96      5768



In [ ]:
# Evaluate the classifier
# print(classification_report(y_test, y_pred))

In [ ]:
# Split your dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust the number of features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize and train the SVM classifier
svm_classifier = SVC(kernel='linear')  # You can try different kernels (linear, rbf, etc.)
svm_classifier.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = svm_classifier.predict(X_test_tfidf)

# Evaluate the classifier
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

    Education       0.95      0.94      0.95      1003
Entertainment       0.93      0.96      0.94      1151
   FoodnDrink       0.95      0.95      0.95       817
     Politics       0.96      0.96      0.96       755
       Sports       0.97      0.96      0.96      1116
   Technology       0.95      0.94      0.94       926

     accuracy                           0.95      5768
    macro avg       0.95      0.95      0.95      5768
 weighted avg       0.95      0.95      0.95      5768



BERT

In [37]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

c:\Users\Izzham Burhan\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Izzham Burhan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-ba

In [38]:
# Tokenize and encode the training and testing data
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, return_tensors='pt')

In [48]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder
label_encoder = LabelEncoder()

train_labels_encoded = label_encoder.fit_transform(X_train)
test_labels_encoded = label_encoder.transform(X_test)

ValueError: y contains previously unseen labels: 'commonwealthgames2022 indian badminton player lakshya sen beats ng tze yong malaysia 1921 219 2116 win mens singles gold victory india records 20th gold cwg2022'

In [47]:
train_dataset = torch.utils.data.TensorDataset(
    train_encodings['input_ids'],  # Input IDs
    train_encodings['attention_mask'],  # Attention mask
    torch.tensor(train_encodings)  # Labels (assuming 'interest' contains numeric labels)
)

ValueError: could not determine the shape of object type 'BatchEncoding'

In [41]:
train_encodings['input_ids']

tensor([[ 101, 3463, 2712,  ...,    0,    0,    0],
        [ 101, 2082, 2045,  ...,    0,    0,    0],
        [ 101, 2292, 2404,  ...,    0,    0,    0],
        ...,
        [ 101, 2824, 4888,  ...,    0,    0,    0],
        [ 101, 3522, 4440,  ...,    0,    0,    0],
        [ 101, 3463, 5297,  ...,    0,    0,    0]])